Please run the below cell only once to install the required libraries. Ignore if already installed.

In [ ]:
pip install openai==0.28 python-dotenv

In [ ]:
# Import all the required libraries
import openai
import os
import time
import imaplib
import email
import re
import json
import email
from email.message import EmailMessage
import smtplib
import json
from string import Template
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.header import Header

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

def get_completion(prompt, model="gpt-3.5-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message["content"]

#Reading the email using imaplib
imap_ssl_host = 'imap.gmail.com'
imap_ssl_port = 993
username = os.getenv('EMAIL_USER')
password = os.getenv('EMAIL_PASS')

#Define search criteria for UNSEEN emails
criteria = 'UNSEEN'

#Creates a search string from criteria.
def search_string(criteria):
    return f'({criteria})'

# Processes the email, extracting its content as a string.
def process_email(data):
    for response_part in data:
        if isinstance(response_part, tuple):
            msg = email.message_from_bytes(response_part[1])
            return msg.as_string()
    return None


mail = imaplib.IMAP4_SSL(imap_ssl_host)
mail.login(username, password)
mail.select('inbox')

processed_email = None

#Initial check to avoid waiting if there are already unread emails
result, data = mail.uid('search', None, search_string(criteria))
uids = [int(s) for s in data[0].split() if s.isdigit()]

# Wait for new unread emails if none are initially found
while not uids:
    print("No new unread emails. Waiting...")
    time.sleep(10)
    mail.select('inbox')
    result, data = mail.uid('search', None, search_string(criteria))
    uids = [int(s) for s in data[0].split() if s.isdigit()]
if uids:
    uid = uids[-1]  # Process the latest unread email
    result, data = mail.uid('fetch', str(uid), '(RFC822)')
    processed_email = process_email(data)
    if processed_email:
        print("New unread email processed. Exiting loop.")
mail.logout()

if processed_email:
    print("Processed email content is available.")

# Processing the read email into a json format
email_content = processed_email 

# Parse the email content into an EmailMessage object
msg = email.message_from_string(email_content)

# Function to get the email body
def get_email_body(msg):
    if msg.is_multipart():
        for part in msg.walk():
            content_type = part.get_content_type()
            content_disposition = str(part.get("Content-Disposition"))

            # Skip any text/plain (txt) attachments
            if content_type == "text/plain" and "attachment" not in content_disposition:
                return part.get_payload(decode=True).decode()  # decode
            elif content_type == "text/html" and "attachment" not in content_disposition:
                return part.get_payload(decode=True).decode()  # to decode
    else:
        # Handle non-multipart emails
        return msg.get_payload(decode=True).decode()

# Extract the body
email_body = get_email_body(msg)

# Regex patterns to find the specific fields
from_pattern = r"From: (.+?) <(.+?)>"
to_pattern = r"To: (.+)"
subject_pattern = r"Subject: (.+)"

# Using the search function to find the first occurrence according to the regex patterns
from_match = re.search(from_pattern, email_content)
to_match = re.search(to_pattern, email_content)
subject_match = re.search(subject_pattern, email_content)

# Extracted fields
from_name = from_email = to_email = subject = ""
if from_match:
    from_name, from_email = from_match.groups()
if to_match:
    to_email = to_match.group(1)
if subject_match:
    subject = subject_match.group(1)

# Form the data into a structured dictionary including the email body/content
email_data = {
    "sent_from_email": from_email,
    "sent_to_email": to_email,
    "subject": subject,
    "name": from_name,
    "email": from_email,
    "body": email_body  # Include the email body/content
}

# Convert the dictionary into a JSON string
email_data_json = json.dumps(email_data, indent=2)
print(email_data_json)

# User's dataset and generating a prompt using the email received
def load_user_data(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    return data

ser_data_json = user_data_json = load_user_data('user_data.json')

def load_and_format_prompt(filename, user_data, email_data):
    with open(filename, 'r') as file:
        prompt_template = file.read()
    
    # Convert JSON data to string and format the prompt
    formatted_prompt = prompt_template.replace('{{user_data_json}}', json.dumps(user_data, indent=2)) \
                                      .replace('{{email_data_json}}', json.dumps(email_data, indent=2))
    return formatted_prompt


def load_and_format_prompt2(filename, email_data):
    with open(filename, 'r') as file:
        prompt_template = file.read()
    
    # Insert actual email data JSON into the prompt template
    formatted_prompt = prompt_template.replace('{{email_data_json}}', json.dumps(email_data, indent=2))
    return formatted_prompt

prompt2 = load_and_format_prompt2('prompt2.txt', email_data_json)

response2 = get_completion(prompt2)
print(response2)

# Using SMTP to send emails
MY_ADDRESS = os.getenv('EMAIL_USER')
PASSWORD = os.getenv('EMAIL_PASS')
s = smtplib.SMTP(host='smtp.gmail.com', port=587)
s.starttls()
s.login(MY_ADDRESS, PASSWORD)

email_data = json.loads(email_data_json)

# Gmail's SMTP server
SMTP_HOST = 'smtp.gmail.com'  
SMTP_PORT = 587

# Send the email if the received email is classified as a collaboration email
if response2.strip() == "True":
    try:
        prompt = load_and_format_prompt('prompt.txt', user_data_json, email_data_json)
        response = get_completion(prompt)
        print(response)

        to_email = email_data['sent_from_email']
        from_email = MY_ADDRESS  # Sender's email address
        subject = f"Re: {email_data['subject']}"  
        body = response

        # Create the email message
        message = MIMEMultipart()
        message['From'] = Header(from_email)
        message['To'] = Header(to_email)
        message['Subject'] = Header(subject)
        message.attach(MIMEText(body, 'plain'))

        # Connect to the SMTP server
        server = smtplib.SMTP(SMTP_HOST, SMTP_PORT)
        server.starttls()
        server.login(MY_ADDRESS, PASSWORD)

        # Send the email
        server.sendmail(from_email, to_email, message.as_string())
        print("Email sent successfully!")
    
    except Exception as e:
        print(f"Error: {e}")

    finally:
        # Terminate the SMTP session
        server.quit()

else:
    print("Email not sent - not a collaboration email.")